<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **BERT (Bidirectional Encoder Representations from Transformers)**

"*BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding*"
https://arxiv.org/pdf/1810.04805.pdf


Desarrollado por Google en 2018, marcó un hito significativo en el campo del NLP. Este modelo se basa en la arquitectura Transformer, introducida en 2017 por Vaswani et al., y es conocido por su capacidad de entender el contexto de las palabras en el texto de una manera más sofisticada que los modelos anteriores. A diferencia de los enfoques anteriores que procesaban el texto de manera secuencial (de izquierda a derecha o viceversa), BERT analiza el texto en ambas direcciones simultáneamente. Esta bidireccionalidad permite a BERT capturar el contexto completo de una palabra, mirando tanto el texto anterior como el siguiente. Esto lo hace particularmente eficaz para entender el significado y la intención detrás de las palabras en oraciones complejas.

BERT se preentrena en un corpus masivo de texto no etiquetado y utiliza dos estrategias principales:

- **Masked Language Model (MLM):** donde se ocultan aleatoriamente palabras del texto y el modelo aprende a predecirlas.

- **Next Sentence Prediction (NSP):** donde el modelo aprende a predecir si una oración es la continuación lógica de otra.

### **1. Masked Language Model (MLM)**

1. **Enmascaramiento de Tokens**:
   - Alrededor del 15% de los tokens en cada secuencia de entrada se seleccionan al azar para ser enmascarados.
   - Estos tokens seleccionados se reemplazan con un token especial `[MASK]`.

2. **Diversificación en el Enmascaramiento**:
   - No todos los tokens seleccionados se enmascaran de la misma manera:
     - Un 80% de las veces, el token seleccionado se reemplaza realmente por `[MASK]`.
     - Un 10% de las veces, se reemplaza con un token aleatorio.
     - En el 10% restante, el token se deja sin cambios.

     Al diversificar el enmascaramiento, se asegura que el modelo no se sobreajuste a los tokens [MASK] y que aprenda a utilizar el contexto para predecir palabras, lo cual es más representativo de cómo se utilizará el modelo en aplicaciones del mundo real.

     Al dejar algunos tokens seleccionados sin cambios, BERT aprende a predecir la idoneidad de las palabras actuales en su contexto original.

     La sustitución con tokens aleatorios ayuda a que el modelo sea robusto frente a entradas inesperadas o ruidosas, mejorando su capacidad para manejar errores o variaciones en los datos de entrada.

3. **Predicción de Tokens Enmascarados**:
   - El objetivo del modelo durante el entrenamiento es predecir los tokens originales de aquellos que han sido enmascarados o alterados.
   - Esto enseña a BERT a entender el contexto y la relación entre las palabras en una secuencia.

### **2. Next Sentence Prediction (NSP)**

1. **Entendimiento de Relaciones entre Oraciones**:
   - NSP es una tarea de clasificación binaria para predecir si una oración B es la continuación lógica de una oración A.

2. **Preparación de Datos de Entrenamiento**:
   - Durante el preentrenamiento, se toman pares de oraciones de un corpus de texto.
   - En aproximadamente el 50% de los casos, la oración B es de hecho la oración que sigue naturalmente a la oración A.
   - En el otro 50%, la oración B es una oración aleatoria del corpus, no relacionada con la oración A.

3. **Uso del Token `[CLS]`**:
   - Para cada par de oraciones, el token `[CLS]` se añade al principio, y un token `[SEP]` se utiliza para separar las dos oraciones.
   - El modelo utiliza la representación del token `[CLS]` para hacer la predicción.

### Importancia del Preentrenamiento

1. **Aprendizaje de Contexto y Relaciones**:
   - MLM y NSP juntos enseñan a BERT a entender tanto el contexto a nivel de palabra (a través de MLM) como las relaciones entre oraciones completas (a través de NSP).


2. **Aplicabilidad en Tareas de NLP**:
   - Una vez preentrenado, BERT puede ser afinado con un conjunto de datos más pequeño para tareas específicas de NLP, como la clasificación de texto, la respuesta a preguntas y el reconocimiento de entidades nombradas.


### **Módulo embedding**

In [ ]:
class BertEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.word_embeddings = nn.Embedding(config.vocab_size, config.emb_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_seq_length, config.emb_size)
        self.token_type_embeddings = nn.Embedding(2, config.emb_size)
        self.LayerNorm = nn.LayerNorm(config.emb_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout)
        
        # position ids (used in the pos_emb lookup table) that we do not want updated through backpropogation
        self.register_buffer("position_ids", torch.arange(config.max_seq_length).expand((1, -1)))

    def forward(self, input_ids, token_type_ids):
        word_emb = self.word_embeddings(input_ids)
        pos_emb = self.position_embeddings(self.position_ids)
        type_emb = self.token_type_embeddings(token_type_ids)

        emb = word_emb + pos_emb + type_emb
        emb = self.LayerNorm(emb)
        emb = self.dropout(emb)

        return emb

Claro, puedo crear un pequeño conjunto de datos inventado con textos breves de noticias en las categorías de deportes, política y cultura. Cada entrada constará de un breve texto seguido de su etiqueta correspondiente.

1. **Texto**: "El equipo nacional de fútbol ganó su partido de clasificación para el mundial con un marcador de 3-1."
   **Etiqueta**: Deportes

2. **Texto**: "La nueva reforma educativa ha sido aprobada por el gobierno, buscando mejorar la calidad en las escuelas públicas."
   **Etiqueta**: Política

3. **Texto**: "El famoso festival de cine de Madrid ha anunciado su selección oficial de películas para este año."
   **Etiqueta**: Cultura

4. **Texto**: "La selección femenina de baloncesto se alzó con el campeonato europeo tras una emocionante final."
   **Etiqueta**: Deportes

5. **Texto**: "El parlamento ha iniciado debates sobre posibles cambios en la legislación ambiental."
   **Etiqueta**: Política

6. **Texto**: "Se inauguró en Barcelona una exposición dedicada a la obra del pintor surrealista Salvador Dalí."
   **Etiqueta**: Cultura

7. **Texto**: "El tenista Carlos Alcaraz se convierte en el número uno del mundo tras su última victoria en París."
   **Etiqueta**: Deportes

8. **Texto**: "La presidenta anunció un nuevo plan de infraestructuras que promete modernizar las carreteras del país."
   **Etiqueta**: Política

9. **Texto**: "El nuevo museo de arte contemporáneo abrirá sus puertas este fin de semana en Valencia."
   **Etiqueta**: Cultura

10. **Texto**: "Histórica medalla de oro para el equipo de natación en el campeonato mundial celebrado en Tokio."
    **Etiqueta**: Deportes

Este conjunto de datos es puramente ilustrativo y puede servir como punto de partida para tareas de clasificación de texto en proyectos de aprendizaje automático o análisis de datos.